<a href="https://colab.research.google.com/github/Salam-35/Plant-Disease-Detection/blob/main/yolov9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from IPython.display import clear_output

In [ ]:
!git clone https://github.com/WongKinYiu/yolov9
%cd yolov9
!pip install -r requirements.txt -q
clear_output()

In [ ]:
!wget -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt
!wget -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt
!wget -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt
!wget -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-e.pt

In [ ]:
import os
import pathlib
import shutil
import json
import yaml
import locale
import argparse

import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# @title Model training config
dataset_path = "/content/drive/MyDrive/dataset_path"           # @param {type:"string"}
experiment_name = "v0.0.1.9"              # @param {type:"string"}
model_name = "gelan-e"  # @param ["yolov9-c", "yolov9-e", "gelan-c", "gelan-e"]
result_dir = "/content/drive/MyDrive/result_path" # @param {type:"string"}
num_epochs = 2         # @param {type:"integer"}
epoch_patience = 50     # @param{type: "integer"}
optimizer = "SGD"       # @param ["SGD", "Adam", "AdamW", "LION"]
image_size = 640        # @param{type: "integer"}]
# learning_rate = 0.01    # @param {type: "number"}
batch_size = 16         # @param {type: "integer"}
conf_threshold = 0.001  # @param {type: "number"}
iou_threshold = 0.7     # @param {type: "number"}
pretrained = True       # @param {type:"boolean"}

weight_name = f"{model_name}.pt"
cfg_name = f"models/detect/{model_name}.yaml"

In [ ]:
data_config = {
    "path": dataset_path,
    "train": "train",
    "val": "val",
    "test": "test",

    "names": {
        0: "fracture",
        1: "periostealreaction",
        2: "pronatorsign",
        3: "softtissue",
    }
}

with open("data.yaml", "w") as f:
    yaml.dump(data_config, f)

In [ ]:
!python train.py \
--batch-size {batch_size} \
--epochs {num_epochs} \
--img {image_size} \
--device 0 \
--min-items 0 \
--data data.yaml \
--weights {model_name}.pt \
--cfg models/detect/{model_name}.yaml \
--hyp hyp.scratch-high.yaml \
--optimizer {optimizer} \
--project {result_dir} \
--name {experiment_name} \
--patience {epoch_patience} \
--exist-ok


2024-03-02 20:25:38.014760: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-02 20:25:38.014830: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-02 20:25:38.016207: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-02 20:25:39.055916: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
train: weights=gelan-e.pt, cfg=models/detect/gelan-e.yaml, data=data.yaml, hyp=hyp.scratch-high.yaml, epochs=2, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bu

# Test

In [ ]:
!python val.py --weights {result_dir}/{experiment_name}/weights/best.pt \
--data data.yaml \
--imgsz {image_size} \
--batch {batch_size} \
--conf {conf_threshold} \
--iou {iou_threshold} \
--device 0 \
--data data.yaml \
--project {result_dir} \
--name {experiment_name} \
--task test \
--exist-ok


val: data=data.yaml, weights=['/content/drive/MyDrive/Promit/TraumaXRay/results/v0.0.1.93/weights/best.pt'], batch_size=16, imgsz=640, conf_thres=0.001, iou_thres=0.7, max_det=300, task=test, device=0, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=/content/drive/MyDrive/Promit/TraumaXRay/results, name=v0.0.1.93, exist_ok=True, half=False, dnn=False, min_items=0
YOLOv5 🚀 v0.1-32-g2ec64d3 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
gelan-c summary: 387 layers, 25230172 parameters, 0 gradients, 101.8 GFLOPs
test: Scanning /content/drive/MyDrive/Promit/TraumaXRay/trauma_dataset/v0.0.1/test/labels.cache... 10 images, 0 backgrounds, 0 corrupt: 100% 10/10 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 1/1 [00:02<00:00,  2.07s/it]
                   all         10         13          0          0          